In [128]:
#Imports
import sys #Python
import sklearn #Machine learning library
import numpy as np #numerical packages in python
import scipy as scp #Another numerical package, unused directly but is implicitly used in sklearn
import pandas as pd #Package for data manipulation and analysis
import matplotlib # plotting library
import os
import time
import random 

# SKlearn imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline

In [129]:
#Load Data
data = pd.read_csv('./full_ray_data.csv')
data.head()

,PathSpecification,CoordinateSystem,SystemScale,TransmitterLocation_1,TransmitterLocation_2,TransmitterLocation_3,ReceiverLocation_1,ReceiverLocation_2,ReceiverLocation_3,LineOfSight,...,pAoD_2,pAoA_1,pAoA_2,pPathLoss,pPhaseShift,AngleOfIncidence_1,AngleOfIncidence_2,AngleOfIncidence_3,AngleOfIncidence_4,UID
0,Locations,Geographic,1,34.06945,-118.4425,112.874983,34.0677,-118.44225,109.824938,1,...,35.26439,-135,-35.26439,93.834212,5.767278,NaN,NaN,NaN,NaN,rayObj463
1,Locations,Geographic,1,34.06945,-118.4425,112.874983,34.0677,-118.44225,109.824938,0,...,35.26439,-135,-35.26439,105.421189,5.575128,85.664137,NaN,NaN,NaN,rayObj464
2,Locations,Geographic,1,34.06945,-118.4425,112.874983,34.0677,-118.44225,109.824938,0,...,35.26439,-135,-35.26439,115.935057,6.153309,84.936750,NaN,NaN,NaN,rayObj465
3,Locations,Geographic,1,34.06945,-118.4425,112.874983,34.0677,-118.44225,109.824938,0,...,35.26439,-135,-35.26439,101.653232,3.874205,80.609775,NaN,NaN,NaN,rayObj466
4,Locations,Geographic,1,34.06945,-118.4425,112.874983,34.0677,-118.44225,109.824938,0,...,35.26439,-135,-35.26439,118.707507,4.005678,81.398592,NaN,NaN,NaN,rayObj467


In [130]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 49 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   PathSpecification       1275 non-null   object 
 1   CoordinateSystem        1275 non-null   object 
 2   SystemScale             1275 non-null   int64  
 3   TransmitterLocation_1   1275 non-null   float64
 4   TransmitterLocation_2   1275 non-null   float64
 5   TransmitterLocation_3   1275 non-null   float64
 6   ReceiverLocation_1      1275 non-null   float64
 7   ReceiverLocation_2      1275 non-null   float64
 8   ReceiverLocation_3      1275 non-null   float64
 9   LineOfSight             1275 non-null   int64  
 10  Interactions            0 non-null      float64
 11  PropagationDelay        1275 non-null   float64
 12  PropagationDistance     1275 non-null   float64
 13  AngleOfDeparture_1      1275 non-null   float64
 14  AngleOfDeparture_2      1275 non-null   

In [131]:
#Preprocess Data
data = data.drop(["PathSpecification"],axis= 1)
data = data.drop(["CoordinateSystem"],axis= 1)
data = data.drop(["SystemScale"],axis= 1)
data = data.drop(["UID"],axis= 1)
data = data.drop(["Interactions"],axis= 1)
data = data.drop(["PathLossSource"], axis=1)
data = data.drop(["ReflectionLocations_1", "ReflectionLocations_2", "ReflectionLocations_3", "ReflectionLocations_4", "ReflectionLocations_5", "ReflectionLocations_6"],axis= 1)
data = data.drop(["ReflectionLocations_7", "ReflectionLocations_8", "ReflectionLocations_9", "ReflectionLocations_10", "ReflectionLocations_11", "ReflectionLocations_12"],axis= 1)
data = data.drop(["AngleOfIncidence_1", "AngleOfIncidence_2", "AngleOfIncidence_3", "AngleOfIncidence_4"], axis=1)
data = data.drop(["pPhaseShift", "pPathLoss", "pPropDelay", "pAoD_1", "pAoD_2", "pAoA_1", "pAoA_2"], axis=1)
data = data.drop("pIsLocked", axis=1)

data = data.drop("Frequency", axis=1) # all same frequency (for now)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   TransmitterLocation_1  1275 non-null   float64
 1   TransmitterLocation_2  1275 non-null   float64
 2   TransmitterLocation_3  1275 non-null   float64
 3   ReceiverLocation_1     1275 non-null   float64
 4   ReceiverLocation_2     1275 non-null   float64
 5   ReceiverLocation_3     1275 non-null   float64
 6   LineOfSight            1275 non-null   int64  
 7   PropagationDelay       1275 non-null   float64
 8   PropagationDistance    1275 non-null   float64
 9   AngleOfDeparture_1     1275 non-null   float64
 10  AngleOfDeparture_2     1275 non-null   float64
 11  AngleOfArrival_1       1275 non-null   float64
 12  AngleOfArrival_2       1275 non-null   float64
 13  NumInteractions        1275 non-null   int64  
 14  PathLoss               1275 non-null   float64
 15  Phas

In [132]:
data.head()

,TransmitterLocation_1,TransmitterLocation_2,TransmitterLocation_3,ReceiverLocation_1,ReceiverLocation_2,ReceiverLocation_3,LineOfSight,PropagationDelay,PropagationDistance,AngleOfDeparture_1,AngleOfDeparture_2,AngleOfArrival_1,AngleOfArrival_2,NumInteractions,PathLoss,PhaseShift,NumReflections,NumDiffractions
0,34.06945,-118.4425,112.874983,34.0677,-118.44225,109.824938,1,6.521530e-07,195.510545,-83.220114,-0.894755,96.779745,0.892993,0,93.834212,5.767278,0,0
1,34.06945,-118.4425,112.874983,34.0677,-118.44225,109.824938,0,6.539812e-07,196.058637,-83.218249,-4.569197,96.777212,-4.104291,1,105.421189,5.575128,0,1
2,34.06945,-118.4425,112.874983,34.0677,-118.44225,109.824938,0,6.541632e-07,196.113200,-82.901520,-3.602422,95.913223,-6.456477,1,115.935057,6.153309,0,1
3,34.06945,-118.4425,112.874983,34.0677,-118.44225,109.824938,0,6.566028e-07,196.844557,-99.232679,-0.888704,99.549910,0.886928,1,101.653232,3.874205,0,1
4,34.06945,-118.4425,112.874983,34.0677,-118.44225,109.824938,0,6.567729e-07,196.895568,-97.103036,-0.888474,100.101723,0.886698,1,118.707507,4.005678,0,1


In [133]:
#Split target and data
data_pathloss = data["PathLoss"]
data_rays = data.drop(["PathLoss"],axis = 1)

#Train test split 
train_raw, test_raw, target, target_test = train_test_split(data_rays, data_pathloss, test_size=0.2, random_state=0)